# Beyond Correlation: Why Likert-Scale Calibration Challenges LLM-as-Participant Designs.



# Code structure: #

In the **first block** of the code, some libraries (such as openai) are imported and the own API-key of OpenAI is defined.

In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "" # insert here OpenAI API key


This is a test, it works! How can I assist you further?


The **second block** of the code contains the prompt engineering functions that we will use later. You will find zero and few shot prompting as example functions. However, only the zero-shot function is applied in the next steps, since few-shot prompting performed poorely in first pilot studies.

In [ ]:
import openai
from openai import OpenAI

# Funktion zur Durchführung von Null-Shot-Prompting
def zero_shot_prompting(task, prompt):
    """
    Führt Null-Shot-Prompting durch, indem ein Prompt an das Sprachmodell gesendet wird,
    ohne dass vorherige Beispiele gegeben werden.

    Args:
    task (str): Die Aufgabe, die durchgeführt werden soll.
    prompt (str): Der Text, der als Eingabe an das Sprachmodell gesendet wird.

    Returns:
    str: Die Antwort des Sprachmodells.
    """
    # Erstellen der Chat-Nachricht und Senden an das Sprachmodell
    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "system", "content": "Tu es un(e) assistant(e)"}, # HIER ÄNDERN, WENN GEWÜNSCHT
            {"role": "user", "content": prompt} # Benutzernachricht mit dem eigentlichen Prompt
        ]
    )
    # Rückgabe des Inhalts der ersten Antwortnachricht des Modells
    return response.choices[0].message.content



# Funktion zur Durchführung von Few-Shot-Prompting
def few_shot_prompting(task, examples, prompt):
    """
    Führt Few-Shot-Prompting durch, indem einige Beispiele gegeben werden,
    bevor der eigentliche Prompt an das Sprachmodell gesendet wird.

    Args:
    task (str): Die Aufgabe, die durchgeführt werden soll.
    examples (list of dict): Eine Liste von Beispielen, wobei jedes Beispiel ein Wörterbuch mit 'input' und 'output' enthält.
    prompt (str): Der Text, der als Eingabe an das Sprachmodell gesendet wird.

    Returns:
    str: Die Antwort des Sprachmodells.
    """
    # Initialisieren der Nachrichtenliste mit einer Systemnachricht
    messages = [{"role": "system", "content": "Tu es un(e) assistant(e)"}] # HIER ÄNDERN, WENN GEWÜNSCHT

    # Hinzufügen der Beispiele zu den Nachrichten
    for example in examples:
        messages.append({"role": "user", "content": example['input']})
        messages.append({"role": "assistant", "content": example['output']})

    # Hinzufügen des eigentlichen Prompts zur Nachrichtenliste
    messages.append({"role": "user", "content": prompt})

    # Erstellen der Chat-Nachricht und Senden an das Sprachmodell
    response = client.chat.completions.create(
        model="o4-mini",
        messages=messages
    )
    # Rückgabe des Inhalts der ersten Antwortnachricht des Modells
    return response.choices[0].message.content


In the **third block** of the code you find code so that you can read in your uploaded data. This will be helpful in presenting the data to the LLM during prompting.

In [ ]:
import pandas as pd
import time
import json
import gc
from psutil import virtual_memory
from datetime import datetime
import random

# Load data
daten = '630similarFrenchWordPair.csv'  # Adjust the filename as needed
df = pd.read_csv(daten)

# Extract specific columns and pack them into a dictionary
columns_to_extract = ['wordPairs', 'PairMeanConcreteness', 'clusters', 'MeanPairSimilarity']
selected_data = df[columns_to_extract].astype(str)
stimuli_dict = selected_data.to_dict(orient="index")

# Create list of items
all_items = [entry['wordPairs'] for entry in stimuli_dict.values()]
#random.shuffle(all_items)
print("The materials for the study are:")

n = len(all_items) // 6

part1 = all_items[0*n : 1*n]
print("list 1:", part1)
part2 = all_items[1*n : 2*n]
part3 = all_items[2*n : 3*n]
part4 = all_items[3*n : 4*n]
part5 = all_items[4*n : 5*n]
part6 = all_items[5*n : 6*n]

The materials for the study are:
list 1: ['abdiquer-abandonner', 'abdomen-ventre', 'abeille-guêpe', 'abolir-annuler', 'absence-manque', 'abstrait-irréel', 'absurde-ridicule', 'acceptation-accord', 'activisme-militantisme', 'addition-somme', 'admirer-célébrer', 'adulte-enfant', 'aérien-vaporeux', 'aéroport-gare', 'affichage-présentation', 'agence-bureau', 'agile-adroit', 'agilité-habileté', 'agitation-troubles', 'agiter-secouer', 'agriculteur-chasseur', 'aigle-faucon', 'album-classeur', 'algèbre-maths', 'alias-pseudonyme', 'allégorie-métaphore', 'alphabet-chiffre', 'altruisme-bonté', 'ambassade-consulat', 'ambition-désir', 'amer-acide', 'amour-adoration', 'ampoule-bougie', 'analogie-exemple', 'anatomie-forme', 'ancre-grappin', 'animal-bête', 'anormal-étrange', 'anxiété-crainte', 'apathie-ennui', 'appartenir-dépendre', 'apprendre-savoir', 'après-futur', 'arbitraire-injuste', 'arbitre-juge', 'architecture-construction', 'argent-monnaie', 'arrivée-approche', 'arrogance-vanité', 'artisanat-

In the **fourth block** we can apply the prompt engineering functions of block 2. We will formulate our prompts in this cell. In this block, we can test different prompting strategies. We prompt the model as many times as we have (or want to have) particpants tu simulate.

In [ ]:
# -----------------------------
# 1. API-Daten
# -----------------------------
API_URL = # insert your API url here
API_KEY = # insert your API key here

def query_model(model_name, prompt_text, temperature=0.3, max_tokens=200):
    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt_text}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    response = requests.post(API_URL, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


# -----------------------------
# 2. Stimuli sind schon geladen
# -----------------------------
# Aus dem vorherigen Block:
# all_items = [...]
stimuli = part5   # HIER nach Bedarf ändern: Hier werden erstmal zum Probieren nur 2 Stimuli gezeigt. Wenn das Prompt steht, bitte ganze Klammer wegmachen!

print("Anzahl der Stimuli:", len(stimuli))


# -----------------------------
# 3. Anzahl der simulierten Probanden
# -----------------------------
n_probanden = 62          # HIER nach Bedarf ändern
model = "mistral-small:24b"   # HIER nach Bedarf ändern

simulation_results = []   # alle Antworten werden hier gespeichert


# -----------------------------
# 4. Simulation starten
# -----------------------------
print("\nSimulation startet...\n")

for proband in range(1, n_probanden + 1):
    print(f"--- Simulierter Proband {proband} ---")

    for idx, stimulus in enumerate(stimuli):
        # Prompt ändern
        prompt = f"""
        Nous allons vous présenter deux mots separées de un -. Votre tâche consiste à réfléchir au sens de ces mots et indiquer sur une échelle en 7 points à quel point vous considérez que ces deux mots sont similaires sémantiquement (qu’ils ont le même sens).
        Si vous considérez que les deux mots de la paire sont très similaires sémantiquement, vous pouvez choisir le point 7 (« tout à fait similaires »). Si vous considérez que les deux mots de la paire sont très différents sémantiquement, vous pouvez choisir le point 1 (« pas du tout similaires »). Vous pouvez utiliser tous les points de l’échelle.
        Par exemple, si la paire présentée est professeur – enseignant, vous pouvez choisir le point 6. Si la paire présentée est lune – route, vous pouvez choisir le point 1. N’hésitez pas à utiliser toute la gamme des chiffres comprise entre 1 et 7 mais ne vous préoccupez pas du nombre de fois que vous avez utilisé un chiffre en particulier tant qu’il fait référence à votre véritable jugement. Il n’y a pas de bonne ou de mauvaise réponse, mais vous devez indiquer seulement le numéro correspondant.
        Veuillez donc répondre le plus spontanément possible. Ne donnez pas d'éxplications. Les mots sont: “{stimulus}”
        """

        antwort = query_model(model, prompt)

        simulation_results.append({
            "proband": proband,
            "stimulus_index": idx,         # An welcher Position der Stimulus währen der Studie vorkam
            "stimulus_text": stimulus,
            "antwort": antwort
        })

        print(f"Antwort zu Item: '{stimulus}': {antwort}")


print("\nSimulation abgeschlossen!")



Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Antwort zu Item: 'passion-vocation': 6
Antwort zu Item: 'pastèque-orange': 4
Antwort zu Item: 'patience-endurance': 6
Antwort zu Item: 'pause-repos': 6
Antwort zu Item: 'pauvreté-besoin': 4
Antwort zu Item: 'paysage-panorama': 6
Antwort zu Item: 'paysan-berger': 5
Antwort zu Item: 'pays-nation': 6
Antwort zu Item: 'pensée-idée': 6
Antwort zu Item: 'penser-réfléchir': 6
Antwort zu Item: 'perceuse-foreuse': 6
Antwort zu Item: 'périr-mourir': 6
Antwort zu Item: 'permettre-autoriser': 6
Antwort zu Item: 'persistance-effort': 6
Antwort zu Item: 'peur-effroi': 6
Antwort zu Item: 'photo-image': 6
Antwort zu Item: 'pinceau-stylo': 4
Antwort zu Item: 'pionnier-fondateur': 6
Antwort zu Item: 'plage-étang': 3
Antwort zu Item: 'planète-astéroïde': 3
Antwort zu Item: 'plasticité-malléabilité': 6
Antwort zu Item: 'plénitude-abondance': 6
Antwort zu Item: 'poche-sac': 4
Antwort zu Item: 'poignée-anse': 4
Antwort zu Item: 'poison-virus

In the **fifth block** we save the results from all test subjects in an Excel and csv file. These files subsequently store the answers of all LLMs-participants.

In [ ]:
# 1. Ursprüngliche Stimuli-Datei laden
df = pd.read_csv("630similarFrenchWordPair.csv")
df = df.rename(columns=lambda c: c.strip())

# 2. simulation_results → DataFrame
sim_df = pd.DataFrame(simulation_results)

# Spalten passend umbenennen
sim_df = sim_df.rename(columns={
    "proband": "Informant",
    "stimulus_index": "Index",
    "stimulus_text": "sentences",
    "antwort": "models_answer"
})

# 3. Merge über target_sent (df) und sentences (sim_df)
merged = df.merge(
    sim_df,
    left_on="wordPairs",
    right_on="sentences",
    how="left"
)

# 4. Überflüssige Spalte entfernen
merged = merged.drop(columns=["sentences"])

# 5. Speichern
merged.to_csv("list5_Lakhzoum_mistral_fewshot_t0.3.csv", index=False)
#merged.to_excel("Lombard_stimuli_mit_Modell_Antworten.xlsx", index=False)

print("Neue Datei erfolgreich erstellt!")

Neue Datei erfolgreich erstellt!
